# Cantera module for Flamelet Generated Manifolds
## module 2: series of flamelets
We are going to run a series of free flames, visualise them and create an FGM for ANSYS Fluent. 
As always, go through this interactive notebook in jupyter (jupyter labs or jupyter notebook) and follow the instructions. You can execute the selected cell one by one using the play button at the top or pressing SHIFT+ENTER.
First we load the necessary libraries.
For help on the commands see https://cantera.org/documentation/

In [ ]:
##%matplotlib inline
%matplotlib qt5

matplotlib inline will embed the figures in your notebook document
if qt5 is installed, figures will be created in an interactive external window

In [2]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt

We are going to compute the solution of a premixed free flame of methane and air, similar to the example on the cantera website

In [4]:
"""
A freely-propagating, premixed methane-air flat flame with unity Lewis transport properties.
"""

# default atmospheric conditions
p = ct.one_atm  # pressure [Pa]
T_unburnt=300   # unburned gas temperature [K]

# numerical parameters
mywidth = 0.03  # total width of the domain in m
myratio = 5
myslope=0.06
mycurve=0.10

# setup parameters
# loglevel = 0 means you will not get any output during the computation
# 1 gives you some output while cantera computes a solution
# >1 gives you a lot of additional output
loglevel = 1  # amount of diagnostic output (0 to 8)

# IdealGasMix object used to compute mixture properties, set to the state of the
# upstream fuel-air mixture
# gri30 is the famous GRI 3.0 mechanism from Berkeley.
gas = ct.Solution('gri30.xml')
gas.TP = T_unburnt,p
# set the gas using temperature T, pressure P, mole fractions X
#reactants = 'H2:1.1, O2:1, AR:5'  # premixed gas composition
#gas.TPX = Tin, p, reactants

# set the gas using equivalence ratio, fuel species and air composition
phi = 0.80
fuel_species = 'CH4'
air_composition='O2:0.21,N2:0.79'
gas.set_equivalence_ratio(phi,fuel_species,air_composition)

In [10]:
# create a free flame on a predefined mesh with a fixed number of points
# We put more points in the middle of the domain in the flame region
# a fixed mesh will make it easier to write an FGM later
x=np.linspace(-0.999,0.999,200)
y=np.arctanh(x) 
y[0]=-3.0
y[len(y)-1]=4.5
y=(y+3.0)*(3/750)
print(y)

[0.         0.00161235 0.00291607 0.0037053  0.00427476 0.00472158
 0.00509003 0.00540405 0.00567804 0.00592136 0.00614044 0.00633987
 0.00652306 0.0066926  0.0068505  0.00699838 0.00713753 0.007269
 0.00739369 0.00751231 0.00762552 0.00773382 0.00783769 0.00793753
 0.00803368 0.00812645 0.00821611 0.00830289 0.00838702 0.00846868
 0.00854805 0.00862528 0.00870051 0.00877387 0.00884547 0.00891543
 0.00898384 0.00905079 0.00911636 0.00918063 0.00924367 0.00930555
 0.00936632 0.00942604 0.00948477 0.00954255 0.00959943 0.00965546
 0.00971067 0.0097651  0.0098188  0.00987178 0.00992409 0.00997576
 0.01002681 0.01007728 0.01012718 0.01017654 0.01022539 0.01027374
 0.01032163 0.01036906 0.01041606 0.01046265 0.01050885 0.01055467
 0.01060013 0.01064524 0.01069003 0.0107345  0.01077867 0.01082255
 0.01086615 0.0109095  0.0109526  0.01099546 0.0110381  0.01108052
 0.01112274 0.01116477 0.01120662 0.0112483  0.01128982 0.01133118
 0.01137241 0.0114135  0.01145447 0.01149533 0.01153608 0.011576

In [6]:
gas.TP = T_unburnt,p
#flame = ct.FreeFlame(gas, width=mywidth)
flame = ct.FreeFlame(gas, grid=y)
#flame.set_refine_criteria(ratio=myratio, slope=myslope, curve=mycurve)
flame.transport_model = 'UnityLewis'
flame.solve(loglevel=loglevel, refine_grid=False, auto=True)


*********** Solving on 200 point grid with energy equation enabled ***********

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     9.492e-06      6.389
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.424e-05      6.534
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.424e-05      6.543
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.602e-05      6.531
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.802e-05      6.511
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     3.041e-05      6.499
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     2.566e-05      6.473
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     2.887

We could have used Cantera's automatic regridding to get a more optimal solution. However, we need a fixed number of points N in progress variable space and a fixed number of points in enthalpy space for our lookup table (else we would not have a nice table with NxM points). The number of points do not have to be equally distributed in the lookup table. 

Can you come up with a strategy to get a flame solution on a 200 point grid, where the grid points are more optimally distributed, taking into account information from the flame?

In [9]:
plt.figure()

#We can globally change the figure size 
plt.rcParams['figure.figsize'] = [8, 4]
plt.xlim(0.008,0.015)
plt.plot(flame.grid,flame.T,'o')
plt.xlabel('Distance [m]')
plt.ylabel('Temperature [K]');


In [18]:
# We are going to create an FGM with different enthalpy levels.
# To increase the enthalpy, we can increase the inlet temperature of the mixture. 
# We are going to create a list of flames with unburnt temperature of T=300,350,400,450,500,550,600,650,700,750

T_unburnt_list = [300,400,500,600]
flamelist = []

# now we loop over the list of unburnt temperatures. This might take a while (1-2 minutes per iteration), 
# so we create some intermediate output to entertain you while waiting...
    
# default atmospheric conditions
p = ct.one_atm  # pressure [Pa]

loglevel = 1  # amount of diagnostic output (0 to 8)

# set the gas using equivalence ratio, fuel species and air composition
phi = 0.80
fuel_species = 'CH4'
air_composition='O2:0.21,N2:0.79'
gas = ct.Solution('gri30.xml')
gas.set_equivalence_ratio(phi,fuel_species,air_composition)


for i,T_unburnt in enumerate(T_unburnt_list):
    print(i," ",T_unburnt)
    gas.TP = T_unburnt,p
    flame = ct.FreeFlame(gas, grid=y)
    flame.transport_model = 'UnityLewis'
    flame.solve(loglevel=loglevel, refine_grid=False, auto=True)
    # append the flame to the flamelist
    flamelist.append(flame)

0   300

*********** Solving on 200 point grid with energy equation enabled ***********

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     9.492e-06      6.389
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.424e-05      6.534
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.424e-05      6.543
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.602e-05      6.531
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.802e-05      6.511
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     3.041e-05      6.499
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     2.566e-05      6.473
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps  

CanteraError: Could not find a solution for the 1D problem

In [17]:
# let's plot some results. First we create an empty figure
plt.figure()
#We can globally change the figure size 
plt.rcParams['figure.figsize'] = [12, 6]
plt.xlim(0.008,0.015)
plt.plot(flamelist[0].grid,flamelist[0].T,label='T=300 K')
plt.plot(flamelist[1].grid,flamelist[1].T,label='T=400 K')
plt.plot(flamelist[2].grid,flamelist[2].T,label='T=500 K')
plt.plot(flamelist[3].grid,flamelist[3].T,label='T=600 K')
plt.xlabel('Distance [m]')
plt.ylabel('Temperature [K]')
# place a semi-transparent legend on the figure at a specified location
plt.legend(framealpha=0.5,frameon=True,loc='upper right')

In [9]:
# first we get a list of all progress variable data, we take CO2
#progvar = [flamelist[0].Y[15].tolist(),flamelist[1].Y[15].tolist(),flamelist[2].Y[15].tolist(),flamelist[3].Y[15].tolist()]
#enthalpy=[flamelist[0].enthalpy_mass.tolist(),flamelist[1].enthalpy_mass.tolist(),flamelist[2].enthalpy_mass.tolist(),flamelist[3].enthalpy_mass.tolist()]
#X,Y = np.meshgrid(progvar,enthalpy)

Z0 = [flamelist[0].net_production_rates[15].tolist(),flamelist[1].net_production_rates[15].tolist(),flamelist[2].net_production_rates[15].tolist(),flamelist[3].net_production_rates[15].tolist()]
Z1 = [flamelist[0].net_production_rates[15].tolist(),flamelist[1].net_production_rates[15].tolist(),flamelist[2].net_production_rates[15].tolist(),flamelist[3].net_production_rates[15].tolist()]
Z2 = [flamelist[0].net_production_rates[15].tolist(),flamelist[1].net_production_rates[15].tolist(),flamelist[2].net_production_rates[15].tolist(),flamelist[3].net_production_rates[15].tolist()]
Z3 = [flamelist[0].net_production_rates[15].tolist(),flamelist[1].net_production_rates[15].tolist(),flamelist[2].net_production_rates[15].tolist(),flamelist[3].net_production_rates[15].tolist()]

progvar=flamelist[0].Y[15]
enthalpy=np.array([flamelist[0].enthalpy_mass[0],flamelist[1].enthalpy_mass[0],flamelist[2].enthalpy_mass[0],flamelist[3].enthalpy_mass[0]])

Z=np.array([flamelist[0].net_production_rates[15]])


In [17]:
print(progvar)

[1.34605179e-17 2.83948522e-17 2.84032486e-17 2.84032595e-17
 2.84032655e-17 2.84032713e-17 2.84032770e-17 2.84032825e-17
 2.84032878e-17 2.84032930e-17 2.84032980e-17 2.84033028e-17
 2.84033076e-17 2.84033122e-17 2.84033166e-17 2.84033210e-17
 2.84033252e-17 2.84033293e-17 2.84033332e-17 2.84033371e-17
 2.84033409e-17 2.84033445e-17 2.84033481e-17 2.84033515e-17
 2.84033549e-17 2.84033582e-17 2.84033613e-17 2.84033644e-17
 2.84033675e-17 2.84033704e-17 2.84033733e-17 2.84033761e-17
 2.84033790e-17 2.84033818e-17 2.84033849e-17 2.84033885e-17
 2.84033933e-17 2.84034006e-17 2.84034133e-17 2.84034375e-17
 2.84034860e-17 2.84035859e-17 2.84037942e-17 2.84042316e-17
 2.84051523e-17 2.84070934e-17 2.84111876e-17 2.84198261e-17
 2.84380544e-17 2.84765192e-17 2.85576876e-17 2.87289658e-17
 2.90903828e-17 2.98529976e-17 3.14621303e-17 3.48573573e-17
 4.20210443e-17 5.71356112e-17 8.90250323e-17 1.56305642e-16
 2.98252598e-16 5.97724217e-16 1.22952359e-15 2.56242236e-15
 5.37438850e-15 1.130662

Traceback (most recent call last):
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/mpl_toolkits/mplot3d/axes3d.py", line 1137, in _button_release
    toolbar = getattr(self.figure.canvas, "toolbar")
AttributeError: 'NoneType' object has no attribute 'canvas'
Traceback (most recent call last):
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/mpl_toolkits/mplot3d/axes3d.py", line 1137, in _button_release
    toolbar = getattr(self.figure.canvas, "toolbar")
AttributeError: 'NoneType' object has no attribute 'canvas'
Traceback (most recent call last):
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/matplotlib/cbook/

In [15]:
#pv=flamelist[0].Y[15]
#enthalpy=flamelist[0].enthalpy_mass
X,Y=np.meshgrid(progvar,enthalpy)
#Z=X*Y+1000
Z = np.array([flamelist[0].T,flamelist[1].T,flamelist[2].T,flamelist[3].T])
print(Z)

[[ 300.          300.          300.         ... 1983.64211891
  1983.64218634 1983.64218634]
 [ 400.          400.          400.         ... 2052.22399928
  2052.22412989 2052.22412989]
 [ 500.          500.          500.         ... 2120.88438251
  2120.88449067 2120.88449067]
 [ 600.          600.          600.         ... 2187.74134117
  2187.74150485 2187.74150485]]


In [16]:
ax3d = plt.axes(projection='3d')
ax3d.plot_surface(X, Y, Z,cmap='plasma')
ax3d.set_title('Surface Plot in Matplotlib')
ax3d.set_xlabel('progvar')
ax3d.set_ylabel('enthalpy')
ax3d.set_zlabel('Z')

plt.show()

Traceback (most recent call last):
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/mpl_toolkits/mplot3d/axes3d.py", line 1137, in _button_release
    toolbar = getattr(self.figure.canvas, "toolbar")
AttributeError: 'NoneType' object has no attribute 'canvas'
Traceback (most recent call last):
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/mpl_toolkits/mplot3d/axes3d.py", line 1137, in _button_release
    toolbar = getattr(self.figure.canvas, "toolbar")
AttributeError: 'NoneType' object has no attribute 'canvas'
Traceback (most recent call last):
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/matplotlib/cbook/

In [18]:
ax3d = plt.axes(projection='3d')
ax3d.plot_wireframe(X, Y, Z,cmap='plasma')
ax3d.set_title('Surface Plot in Matplotlib')
ax3d.set_xlabel('progvar')
ax3d.set_ylabel('enthalpy')
ax3d.set_zlabel('Z')

plt.show()

Traceback (most recent call last):
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/mpl_toolkits/mplot3d/axes3d.py", line 1137, in _button_release
    toolbar = getattr(self.figure.canvas, "toolbar")
AttributeError: 'NoneType' object has no attribute 'canvas'
Traceback (most recent call last):
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/mpl_toolkits/mplot3d/axes3d.py", line 1137, in _button_release
    toolbar = getattr(self.figure.canvas, "toolbar")
AttributeError: 'NoneType' object has no attribute 'canvas'
Traceback (most recent call last):
  File "/home/nijso/anaconda3/envs/envcantera/lib/python3.7/site-packages/matplotlib/cbook/

In [ ]:
plt.figure()
#We can globally change the figure size 
plt.rcParams['figure.figsize'] = [8, 4]
plt.xlim(0.008,0.015)
plt.plot(flamelist[0].grid,flamelist[0].T,label='T=300 K')
plt.plot(flamelist[1].grid,flamelist[1].T,label='T=400 K')
plt.plot(flamelist[2].grid,flamelist[2].T,label='T=500 K')
plt.plot(flamelist[3].grid,flamelist[3].T,label='T=600 K')
plt.xlabel('Distance [m]')
plt.ylabel('Mole Fraction [-]')
# place a semi-transparent legend on the figure at a specified location
plt.legend(framealpha=0.5,frameon=True,loc='upper right')

We are going to create an FGM for use in Fluent
First we need to create the header of the file with all the info on the stored data.
Then, we write the FGM data as column data in the ASCII file

In [244]:
# we are going to write the solution as an FGM to a datafile

# points in progress variable direction
pvpoints = len(f.X[0])
# points in enthalpy direction
enthpoints = 2

strdatasize=str(pvpoints) + " " + str(enthpoints) + " \n"
strgridpower="1.0 1.0 \n"
strpoints=str(pvpoints) + " " + str(enthpoints)
print(strpoints)

fgmheader=["[FGM] \n",
           "2DRLR \n",
           "[DIMENSION] \n",
           "2 \n",
           "[CVDEFINITION] \n",
           "2 \n",
           "CO2  1.0 0.0 \n",
           "ENTHALPY 0.0 1.0 \n",
           "[END] \n",
           "[DATASIZE] \n", 
           strdatasize, 
           "[GRIDPOWER] \n", 
           strgridpower,
           "VARIABLES \n",]

varlist=["CV1 \n",
         "CV2 \n",
         "SOURCE_CV1 \n",
         "SOURCE_CV2 \n",
         "DIFF_CV1 \n",
         "DIFF_CV2 \n",
         "DENSITY \n",
         "TEMPERATURE \n",
         "CP \n",
         "CONDUCTIVITY \n", 
         "VISCOSITY \n", 
         "HEATRELEASE \n"]

print(varlist)

for i, specie in enumerate(gas.species()):
    varlist.append(str(specie).replace('<Species ','').replace('>','')+" \n")

varlist.append("ENTHALPY \n")
print(varlist)

# number of variables
numvar = len(varlist)
print(numvar)

fgmheader.append(str(numvar)+" \n")
fgmheader = fgmheader + varlist
fgmheader.append("[END] \n")
fgmheader.append("[DATA] \n")

print(fgmheader)
CP=np.array(f.cp_mass)
CONDUCTIVITY=np.array(f.thermal_conductivity)
DENSITY=np.array(f.density)
DIFF = CONDUCTIVITY/CP
DIFF=DIFF/DENSITY

# index to the species that acts as a progress variable
iPV=5
ENTHALPY = f.enthalpy_mass
# first control variable: progress variable
CV1=f.Y[iPV]
# second control variable: enthalpy
CV2=ENTHALPY

SOURCE_CV1 = f.net_production_rates[iPV]
SOURCE_CV2 = 0
CONDUCTIVITY=np.array(f.thermal_conductivity)
CP=np.array(f.cp_mass)
DENSITY=np.array(f.density)
DIFF_CV1 = CONDUCTIVITY/CP
DIFF_CV1 = DIFF_CV1/DENSITY
DIFF_CV2 = DIFF_CV1
VISCOSITY=f.viscosity
TEMPERATURE=f.T
HEATRELEASE=f.heat_release_rate

# which species do we want to write to the FGM?
SPECIES1 = f.X[gas.species_index("H2O")]
SPECIES2 = f.X[gas.species_index("CH4")]
SPECIES3 = f.X[gas.species_index("O2")]
SPECIES4 = f.X[gas.species_index("H2")]
SPECIES5 = f.X[gas.species_index("CO")]
SPECIES6 = f.X[gas.species_index("CO2")]
SPECIES7 = f.X[gas.species_index("OH")]
SPECIES8 = f.X[gas.species_index("CH")]
SPECIES9 = f.X[gas.species_index("NO")]

import csv
# open the file for writing, create if file does not exist, overwrite if file exists
fgmfile = open("fgm.dat","w",newline='')

# write the header
fgmfile.writelines(fgmheader)

# write the data, loop over the enthalpy points
# for i in range
# write the data, loop over the progress variable points
for i in range(0,pvpoints):
    fgmline = [i,CV1[i],CV2[i],SOURCE_CV1[i],SOURCE_CV2,DIFF_CV1[i],DIFF_CV2[i],DENSITY[i],TEMPERATURE[i],CP[i],VISCOSITY[i],HEATRELEASE[i],SPECIES1[i],SPECIES2[i],SPECIES3[i],SPECIES4[i],SPECIES5[i],SPECIES6[i],SPECIES7[i],SPECIES8[i],SPECIES9[i],ENTHALPY[i]]
    #fgmline = [str(item)+' ' for item in fgmlist]
    #fgmline.append("\n")
    #print(fgmline
    #fgmfile.writelines(line)
    wr=csv.writer(fgmfile,delimiter=' ',lineterminator='\n')
    wr.writerow(item for item in fgmline)
    
fgmfile.write("[END]")
fgmfile.close()

173 2
['CV1 \n', 'CV2 \n', 'SOURCE_CV1 \n', 'SOURCE_CV2 \n', 'DIFF_CV1 \n', 'DIFF_CV2 \n', 'DENSITY \n', 'TEMPERATURE \n', 'CP \n', 'CONDUCTIVITY \n', 'VISCOSITY \n', 'HEATRELEASE \n']
['CV1 \n', 'CV2 \n', 'SOURCE_CV1 \n', 'SOURCE_CV2 \n', 'DIFF_CV1 \n', 'DIFF_CV2 \n', 'DENSITY \n', 'TEMPERATURE \n', 'CP \n', 'CONDUCTIVITY \n', 'VISCOSITY \n', 'HEATRELEASE \n', 'H2 \n', 'H \n', 'O \n', 'O2 \n', 'OH \n', 'H2O \n', 'HO2 \n', 'H2O2 \n', 'C \n', 'CH \n', 'CH2 \n', 'CH2(S) \n', 'CH3 \n', 'CH4 \n', 'CO \n', 'CO2 \n', 'HCO \n', 'CH2O \n', 'CH2OH \n', 'CH3O \n', 'CH3OH \n', 'C2H \n', 'C2H2 \n', 'C2H3 \n', 'C2H4 \n', 'C2H5 \n', 'C2H6 \n', 'HCCO \n', 'CH2CO \n', 'HCCOH \n', 'N \n', 'NH \n', 'NH2 \n', 'NH3 \n', 'NNH \n', 'NO \n', 'NO2 \n', 'N2O \n', 'HNO \n', 'CN \n', 'HCN \n', 'H2CN \n', 'HCNN \n', 'HCNO \n', 'HOCN \n', 'HNCO \n', 'NCO \n', 'N2 \n', 'AR \n', 'C3H7 \n', 'C3H8 \n', 'CH2CHO \n', 'CH3CHO \n', 'ENTHALPY \n']
66
['[FGM] \n', '2DRLR \n', '[DIMENSION] \n', '2 \n', '[CVDEFINITION] \n', '

[<Species H2>,
 <Species H>,
 <Species O>,
 <Species O2>,
 <Species OH>,
 <Species H2O>,
 <Species HO2>,
 <Species H2O2>,
 <Species C>,
 <Species CH>,
 <Species CH2>,
 <Species CH2(S)>,
 <Species CH3>,
 <Species CH4>,
 <Species CO>,
 <Species CO2>,
 <Species HCO>,
 <Species CH2O>,
 <Species CH2OH>,
 <Species CH3O>,
 <Species CH3OH>,
 <Species C2H>,
 <Species C2H2>,
 <Species C2H3>,
 <Species C2H4>,
 <Species C2H5>,
 <Species C2H6>,
 <Species HCCO>,
 <Species CH2CO>,
 <Species HCCOH>,
 <Species N>,
 <Species NH>,
 <Species NH2>,
 <Species NH3>,
 <Species NNH>,
 <Species NO>,
 <Species NO2>,
 <Species N2O>,
 <Species HNO>,
 <Species CN>,
 <Species HCN>,
 <Species H2CN>,
 <Species HCNN>,
 <Species HCNO>,
 <Species HOCN>,
 <Species HNCO>,
 <Species NCO>,
 <Species N2>,
 <Species AR>,
 <Species C3H7>,
 <Species C3H8>,
 <Species CH2CHO>,
 <Species CH3CHO>]

In [257]:
dir(gas)

['DP',
 'DPX',
 'DPY',
 'HP',
 'HPX',
 'HPY',
 'ID',
 'P',
 'P_sat',
 'SP',
 'SPX',
 'SPY',
 'SV',
 'SVX',
 'SVY',
 'T',
 'TD',
 'TDX',
 'TDY',
 'TP',
 'TPX',
 'TPY',
 'T_sat',
 'UV',
 'UVX',
 'UVY',
 'X',
 'Y',
 '__call__',
 '__class__',
 '__copy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_check_kinetics_species_index',
 '_check_phase_index',
 '_check_reaction_index',
 '_full_states',
 '_init_cti_xml',
 '_init_parts',
 '_references',
 'activities',
 'activity_coefficients',
 'add_reaction',
 'add_species',
 'atomic_weight',
 'atomic_weights',
 'basis',
 'binary_diff_coeffs',
 'chemical_potentials',
 'concentrations',
 'cp',
 'cp_mass',
 'cp_mole',
 'cr

In [251]:
gas.species().index("Species O2>")

ValueError: 'Species O2>' is not in list

In [269]:
gas.species_index("H2O")

5

In [270]:
gas.n_species

53